# Scheduling 
Scheduling problem is in interesting subsection in the domain of operations research. It can arise in many different places like in manufacturing (processing sequence of different orders) or in school/college (timetabling) or public transportation timetabling. With covid we are not venturing out much but then also solution of this problem is very useful, as it keeps us well fed :) How you ask, here comes the problem of driver scheduling for food delivery companies.

# driver timetabling
So for delivering food companies like swiggy, zomato etc. need to do driver timetabling. For the sake of simplicity we can assume simpler version of it,**with business rules**:

<ol>
<li>Planning is done on every alternate fridays for next 2 weeks with inputs from driver. This inputs from driver are covered in following points.</li>
<li>There is fixed pool of drivers available which operates in only familiar routes (much like bus/train drivers)</li>
<li>As typically day operations last for more than 16 hours so there are 2 shifts; let's say them day shift and night shift. For each route we need 1 driver per shift. So total 2 driver per route per day.</li>
<li> A driver can only do one shift per day as per labour rules.</li>
<li>A day shift after previous day's night shift is possible but should be avoided as much as possible</li>
<li>As drivers are mostly freelancers/students so they give their preference input for work day and shifts. Obviously honoring such preference makes driver happy and more engaged.</li>
<li>Drivers also plan their vacation days in advance and provide us input. We must not plan the driver on those days.</li>
<li>Drivers also provide their preferred days off. If we must plan the driver on these days then corresponding cost/penalty will be incurred.</li>
<li>Doing too many night shifts is known to cause resentment among drivers. Hence in planning should try to not more than 5 night shift to each driver for entire planning period of 14 days.</li>
<li>Also planning should be done to give not more than 3 consecutive night shifts.</li>
</ol>

So this is a very simple problem that one can solve. So far no math but rather a business problem.

In data science world business problem almost always comes in plain english like the one mentioned above. There can be many more complexities possible in this problem definition but lets start with this simple problem. 


# Towards mathematical formulation
First point which one can understand; after reading above problem description is, there are many combinations possible here. All driver-day-shift-route pair are possible combinations, among which we have to choose a combination which <em>satisfies above business rules as much as possible</em>. Operations research/combinatorial optimization hence comes handy for this type of problem.

Now to the mention in previous paragraph about the combination which <em>satisfies above business rules as much as possible</em>. How do we translate that to numbers/formula?

So for mathematical formulation one needs to understand the problem deeply, keeping an eye on possible numerical translation of the problem. What's a numerical translation, you ask? Its basically understanding which of the above business rules are binding and could not violated? which of those could be voilated albeit with penalty? If there is a penalty for violation of business rules, whats relative importance? 

We can discuss these questions with business stakeholders and come up with relative importance numbers. For this case lets assume following numerical rewards/penalty:
<ol>
<li>Respect a shift preference: +3</li>
<li>Respect a day off preference: +4</li>
<li>For each long rest, 3 or more consecutive days off: +5</li>
<li>For each unassigned shift: -30</li>
<li>For each night shift followed by an early shift for a driver: -20</li>
<li>For each night shift after third consecutive late shift assigned to a single driver: -10</li>
<li>For every night shift assigned to a driver after the 5th: -20</li>
</ol>

With this information we are ready to build our simulation for this case. Lets start with some code:

In [ ]:
#Importing usual data processing libraries
import numpy as np
import pandas as pd
from datetime import datetime
import random

#Global variables and set seed
NUM_ROUTE = 3
NUM_DRIVER = 8
NUM_DAYS = 14 #2 weeks planning 
np.random.seed(42)

In [ ]:
qualified_route = np.where(np.random.rand(NUM_DRIVER, NUM_ROUTE) > 0.6, 1, 0)
#sample data for mandatory day off
forced_day_off = np.where(np.random.rand(NUM_DRIVER, NUM_DAYS) > 0.7, 1, 0)
#logical negation of mandatory off days
not_forced_day_off = np.logical_not(mandatory_vacation_days).astype(int)
#preferred days for vacation. These are generated randomly and then mandatory off days were removed from output
pref_day_off = np.multiply(np.where(np.random.rand(NUM_DRIVER, NUM_DAYS) > 0.6, 1, 0), not_mandatory_vacation_days)
#from not mandatory off days, some days are selected randomly. In these days randomly first or second shift off preference was sampled. 
pref_work_shift = np.multiply(np.multiply(np.where(np.random.rand(NUM_DRIVER, NUM_DAYS) > 0.5, 0, 1), not_mandatory_vacation_days),
np.random.choice([1,2],mandatory_vacation_days.shape))

#Handy loop constructs
shifts = range(2)
routes = range(NUM_ROUTE)
days = range(NUM_DAYS)
drivers = range(NUM_DRIVER)

For example `mandatory_vacation_days` looks like following, on column we have different days and on rows we have drivers (so first driver is taking off 2 days):

In [ ]:
mandatory_vacation_days

As explained above a potential solution could be expressed as combination of day-shift-route-driver. Hence making such a `dict` with binary value could be used for storing solution. Value 1 will indicate presence of such combination in solution and 0 absence. So:

In [ ]:
# creating schedule dict with value as 0
schedule = {k: 0 for k in list(
    itertools.product(days, shifts, routes, drivers))}